## Image-NetのラベルTree構造を把握し、統合を行う

In [2]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 34.8 MB/s eta 0:00:00


In [50]:
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
df = pd.read_csv("/workspace/Moon_Pattern_Inference/data/ImageNet-S_label.csv")
lst_names=df["folder"].tolist()
df.head()

,folder,id,folder.1,name
0,n01440764,1,n01440764,"tench, Tinca tinca"
1,n01443537,2,n01443537,"goldfish, Carassius auratus"
2,n01484850,3,n01484850,"great white shark, white shark, man-eater, man..."
3,n01491361,4,n01491361,"tiger shark, Galeocerdo cuvieri"
4,n01494475,5,n01494475,"hammerhead, hammerhead shark"


In [36]:
wordnet.synset_from_pos_and_offset('n', 1494475).hypernym_paths()

[[Synset('entity.n.01'),
  Synset('physical_entity.n.01'),
  Synset('object.n.01'),
  Synset('whole.n.02'),
  Synset('living_thing.n.01'),
  Synset('organism.n.01'),
  Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('aquatic_vertebrate.n.01'),
  Synset('fish.n.01'),
  Synset('cartilaginous_fish.n.01'),
  Synset('elasmobranch.n.01'),
  Synset('shark.n.01'),
  Synset('hammerhead.n.03')]]

In [39]:
[i for i in lst_names if not "n0" in i ]

['n10565667',
 'n11879895',
 'n11939491',
 'n12057211',
 'n12144580',
 'n12267677',
 'n12620546',
 'n12768682',
 'n12985857',
 'n12998815',
 'n13037406',
 'n13040303',
 'n13044778',
 'n13052670',
 'n13054560',
 'n13133613',
 'n15075141']

In [43]:
hierarchy=[]
for name in lst_names:
    h=[]
    number=int(name.split("n")[1])
    syn = wordnet.synset_from_pos_and_offset('n', number)
    for hyp in syn.hypernym_paths():
        hh=[]
        for path in hyp:
            hh.append(path.lemma_names()[0])
        h.append(hh)
    hierarchy.append(h)

In [51]:
c=collections.Counter([len(i) for i in hierarchy])
print(c)

Counter({1: 626, 2: 269, 3: 16, 4: 5, 5: 3})


In [52]:
c=collections.Counter([len(v) for i in hierarchy for v in i])
print(c)

Counter({10: 238, 11: 185, 12: 183, 9: 135, 13: 116, 14: 86, 15: 77, 17: 64, 8: 58, 16: 49, 18: 37, 7: 13, 19: 4, 5: 2})


In [54]:
[i for i in hierarchy if len(i)==5][0]

[['entity',
  'physical_entity',
  'object',
  'whole',
  'artifact',
  'instrumentality',
  'container',
  'wheeled_vehicle',
  'self-propelled_vehicle',
  'armored_vehicle',
  'tank'],
 ['entity',
  'physical_entity',
  'object',
  'whole',
  'artifact',
  'instrumentality',
  'conveyance',
  'vehicle',
  'wheeled_vehicle',
  'self-propelled_vehicle',
  'armored_vehicle',
  'tank'],
 ['entity',
  'physical_entity',
  'object',
  'whole',
  'artifact',
  'instrumentality',
  'conveyance',
  'vehicle',
  'military_vehicle',
  'tank'],
 ['entity',
  'physical_entity',
  'object',
  'whole',
  'artifact',
  'instrumentality',
  'container',
  'wheeled_vehicle',
  'self-propelled_vehicle',
  'tracked_vehicle',
  'tank'],
 ['entity',
  'physical_entity',
  'object',
  'whole',
  'artifact',
  'instrumentality',
  'conveyance',
  'vehicle',
  'wheeled_vehicle',
  'self-propelled_vehicle',
  'tracked_vehicle',
  'tank']]

In [56]:
[v for i in hierarchy for v in i if len(v)==5]


[['entity', 'abstraction', 'communication', 'sign', 'street_sign'],
 ['entity', 'physical_entity', 'object', 'geological_formation', 'cliff']]

最も数が多いtreeに限定する

In [68]:
hierarchy=[]
for name in lst_names:
    h=[]
    number=int(name.split("n")[1])
    syn = wordnet.synset_from_pos_and_offset('n', number)
    paths=syn.hypernym_paths()
    if len(paths)==0:
        hyp=paths[0]
    else:
        temp=[len(i) for i in paths]
        hyp=paths[temp.index(max(temp))]
    for path in hyp:
        h.append(path.lemma_names()[0])
    hierarchy.append(h)

In [71]:
!pip install anytree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 8.4 MB/s eta 0:00:00


In [78]:
from anytree import Node, AnyNode, RenderTree

In [87]:
root = Node("entity")
dict_nodes = {}
dict_nodes["entity"] = root
for tree in hierarchy:
    for name in tree:
        if name not in dict_nodes:
            parent = dict_nodes[tree[tree.index(name)-1]]
            dict_nodes[name] = Node(name, parent=parent)

In [89]:
for i, (pre, fill, node) in enumerate(RenderTree(dict_nodes["entity"])):
    print("%s%s" % (pre, node.name))
    if i==30:
        break

entity
├── physical_entity
│   ├── object
│   │   ├── whole
│   │   │   ├── living_thing
│   │   │   │   └── organism
│   │   │   │       ├── animal
│   │   │   │       │   ├── chordate
│   │   │   │       │   │   └── vertebrate
│   │   │   │       │   │       ├── aquatic_vertebrate
│   │   │   │       │   │       │   └── fish
│   │   │   │       │   │       │       ├── bony_fish
│   │   │   │       │   │       │       │   └── teleost_fish
│   │   │   │       │   │       │       │       ├── soft-finned_fish
│   │   │   │       │   │       │       │       │   ├── cypriniform_fish
│   │   │   │       │   │       │       │       │   │   └── cyprinid
│   │   │   │       │   │       │       │       │   │       ├── tench
│   │   │   │       │   │       │       │       │   │       └── goldfish
│   │   │   │       │   │       │       │       │   ├── eel
│   │   │   │       │   │       │       │       │   └── salmonid
│   │   │   │       │   │       │       │       │       └── salmon
│   │   │ 

In [91]:
for i, (pre, fill, node) in enumerate(RenderTree(dict_nodes["animal"])):
    print("%s%s" % (pre, node.name))
    if i==30:
        break

animal
├── chordate
│   └── vertebrate
│       ├── aquatic_vertebrate
│       │   └── fish
│       │       ├── bony_fish
│       │       │   └── teleost_fish
│       │       │       ├── soft-finned_fish
│       │       │       │   ├── cypriniform_fish
│       │       │       │   │   └── cyprinid
│       │       │       │   │       ├── tench
│       │       │       │   │       └── goldfish
│       │       │       │   ├── eel
│       │       │       │   └── salmonid
│       │       │       │       └── salmon
│       │       │       │           └── coho
│       │       │       ├── spiny-finned_fish
│       │       │       │   ├── percoid_fish
│       │       │       │   │   ├── butterfly_fish
│       │       │       │   │   │   └── rock_beauty
│       │       │       │   │   └── damselfish
│       │       │       │   │       └── anemone_fish
│       │       │       │   ├── scorpaenoid
│       │       │       │   │   └── scorpaenid
│       │       │       │   │       └── lionfish
│       │